In [ ]:
import numpy as np
np.random.seed(42)

import os
import glob
# the glob module is used to retrieve files/pathnames matching a specified pattern

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from landlab.components import DrainageDensity
from landlab import imshow_grid
import imageio

from terrainbento import Basic

import holoviews as hv
hv.notebook_extension('matplotlib')

We begin by creating an example user defined output writer function. When a terrainbento model runs the `OutputWriter` function, it will pass the model instance to the output writer. Thus the output writer function must be able to make all required calculations and plots based on the model instance. It is almost always the case that a user will want to write some sort of output -- typically a text file or plot -- each time the output writer is run. 

A user has the option of appending the output writer output to a file or using the model attributes `model.model_time` or `model.iteration` in the output file name to distinguish it from other files.  



In [ ]:
def name_of_output_writer_function(model):
    # put lines of code here that use the model object to calculate outputs
    # for example, mean elevation
    val = model.z.mean()

    # write output, option 1: unique files per timestep
    with open('name_of_file.' + str(int(model.iteration)) + '.txt', 'w') as f:
        f.write(str(val))

    # write output, option 2: append values to same file
    with open('name_of_file.txt', 'a') as f:
        f.write(
            str(model.iteration) + ', ' + str(model.model_time) + ', ' +
            str(val) + '\n')

Here we will define an output writer called `write_drainage_density`. Not surprisingly, it calculates and writes out the drainage density of a model domain. The **DrainageDensity** component has two options for identifying where channels are located -- either a user-specified channel mask or a slope-area threshold. We will use this latter option. [See also](https://landlab.readthedocs.io/en/master/reference/components/drainage_density.html)

In [ ]:
def write_drainage_density(model):
    dd = DrainageDensity(model.grid,
                         area_coefficient=0.1,
                         slope_coefficient=1.0,
                         area_exponent=0.5,
                         slope_exponent=0.6,
                         channelization_threshold=0.35)
    mean_drainage_density = dd.calculate_drainage_density()
    if np.isinf(mean_drainage_density):
        mean_drainage_density = 0.0
    fname = 'drainage_density.txt'
    if os.path.exists(fname) is False:
        with open(fname, 'w') as f:
            f.write('model_time,drainage_density\n')

    with open(fname, 'a') as f:
        f.write(
            str(model.model_time) + ', ' + str(mean_drainage_density) + '\n')

    del dd

Next we construct the parameter dictionary we will use to run the model. 

In [ ]:
basic_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 1000,
        "stop": 2e5
    },

    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (25, 40),
            {
                "xy_spacing": 40
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {
            "modify_core_nodes": True,
            "lowering_rate": -0.0005
        }
    },
    # Parameters that control output.
    "output_interval": 1e4,
    "save_first_timestep": True,
    "output_prefix": "model_basic_output_intro_ow1",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.1,
}

Unline the boundary handlers described in [this tutorial](http://localhost:8888/notebooks/example_usage/introduction_to_boundary_conditions.ipynb), we pass the output writer to terrainbento as a keyword argument. 

After instantiating the model, we run it. 

In [ ]:
basic = Basic.from_dict(basic_params,
                        output_writers={"function": [write_drainage_density]})
basic.run()

First we will make a plot of the topography. We have evolved some nice valleys and ridges. 

In [ ]:
imshow_grid(basic.grid, basic.z)

Now that we've run the model we will compile all of the output into a dataframe and plot it. 

Next we plot the channel mask from the last timestep. It is stored in an at-node grid field called `chanel__mask`.

In [ ]:
channel__mask = basic.grid.at_node['channel__mask'].astype(int)
imshow_grid(basic.grid, channel__mask)

Our mask identifies the channel areas. 

Next we will read in the output we wrote to the file `drainage_density.txt` using [pandas](https://pandas.pydata.org) and remove the files created by running the model. In your usage you probably won't want to remove these files, but we are doing it so that running the model won't create lots of files in your directory. 

In [ ]:
df = pd.read_csv('drainage_density.txt')
basic.remove_output_netcdfs()
os.remove('drainage_density.txt')

We can look at what is inside of the dataframe. 

In [ ]:
df.head()

In [ ]:
df.columns

This dataframe shows us the history of drainage density through time. Next we will plot it. 

In [ ]:
plt.figure()
plt.plot(df.model_time, df.drainage_density)
plt.xlabel('Model Time [years]')
plt.ylabel('Drainage Density')
plt.show()

## Challenge: make a plot of the average elevation of the LEM through time

#### Step 1: Create a function to calculate the average elevation

In [ ]:
def write_avg_el(model):
    avg = model.z.mean()    
    
    fname = 'mean_elevation.txt'
    if os.path.exists(fname) is False:
        with open(fname, 'w') as f:
            f.write('model_time,avg\n')

    with open(fname, 'a') as f:
        f.write(
            str(model.model_time) + ', ' + str(avg) + '\n')

    del avg

#### Step 2 run the model 

Next we construct the parameter dictionary we will use to run the model. 

In [ ]:
basic_params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 1000,
        "stop": 5e5
    },

    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (25, 40),
            {
                "xy_spacing": 40
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {
            "modify_core_nodes": True,
            "lowering_rate": -0.0005
        }
    },
    # Parameters that control output.
    "output_interval": 1e4,
    "save_first_timestep": True,
    "output_prefix": "model_basic_output_intro_avg",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": 0.0001,
    "m_sp": 0.5,
    "n_sp": 1.0,
    "regolith_transport_parameter": 0.01,
}

Unline the boundary handlers described in [this tutorial](http://localhost:8888/notebooks/example_usage/introduction_to_boundary_conditions.ipynb), we pass the output writer to terrainbento as a keyword argument. 

After instantiating the model, we run it. 

In [ ]:
basic2 = Basic.from_dict(basic_params,
                        output_writers={"function": [write_avg_el]})
basic2.run()

First we will make a plot of the topography. We have evolved some nice valleys and ridges. 

In [ ]:
imshow_grid(basic2.grid, basic2.z)

In [ ]:
df = pd.read_csv('mean_elevation.txt')
df.head()

In [ ]:
plt.figure()
plt.scatter(df.model_time, df.avg)
plt.xlabel('Model Time [years]')
plt.ylabel('Mean elevation')
plt.show()

In [ ]:
ds2 = basic2.to_xarray_dataset(time_unit='years', space_unit='meters')

In [ ]:
hvds_topo2 = hv.Dataset(ds2.topographic__elevation)
topo2 = hvds_topo2.to(hv.Image, ['x', 'y'],
                    label='Basic').options(interpolation='bilinear',
                                           cmap='viridis',
                                           colorbar=True)
topo2.opts(fontsize={
    'title': 10, 
    'labels': 10, 
    'xticks': 10, 
    'yticks': 10,       
    'cticks': 10,
})
topo2

# Let's make a movie
First, we make an image for each output interval.

In [ ]:
from landlab import imshow_grid

filenames = []
curT = 0
for i in range(ds2.topographic__elevation.shape[0]):
    curT+=basic2.output_interval
    filename = "temp_output."+str(i)+".png"
    imshow_grid(basic2.grid, ds2.topographic__elevation.values[i, :, :], cmap="viridis", limits=(0, 25), output=filename,
               plot_name=str(curT) + ' model years')
    filenames.append(filename)    


Finally we compile the images into a gif.

In [ ]:
with imageio.get_writer("terrainbento_example.gif", mode="I") as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)
        os.remove(filename)

Delete data

In [ ]:
basic2.remove_output_netcdfs()
os.remove('mean_elevation.txt')

## Next Steps

- We recommend you review the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/).

- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](Introduction_to_terrainbento.ipynb) 
    
    2) [Introduction to boundary conditions in terrainbento](introduction_to_boundary_conditions.ipynb)
    
    3) **This Notebook**: [Introduction to output writers in terrainbento](introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) [Basic](../coupled_process_elements/model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](../coupled_process_elements/model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](../coupled_process_elements/model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](../coupled_process_elements/model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](../coupled_process_elements/model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values